In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
import seaborn as sns
import matplotlib.pyplot as plt
from geodatasets import get_path
from function_06 import load_data_with_delimiters
from shapely.geometry import Point
import folium
from folium.plugins import HeatMap
import re
import pyproj
from folium.plugins import MarkerCluster


In [4]:
file_info = [
    ("../data/cleaned/df_wea.csv", ','),    
    ("../data/cleaned/gdf_joined.csv", ','),
    ("../data/cleaned/bike_data.csv", ',')

]

dfs = load_data_with_delimiters(file_info)

df_weather = dfs[0]
df_accidents = dfs[1]
bike_data= dfs[2]
display(df_weather.head())
display(df_accidents.head())
display(bike_data.head())

,Unnamed: 0,date,temp_C,precip_mm,wind_avg_ms,sun_hours,humidity_pct,visibility_min_m,visibility_max_m
0,24107,2017-01-01,1.6,0.6,5.3,0.0,97.0,200.0,3800.0
1,24108,2017-01-02,3.2,2.0,3.3,4.6,83.0,2800.0,32000.0
2,24109,2017-01-03,5.7,0.4,7.2,0.7,85.0,11000.0,25000.0
3,24110,2017-01-04,5.9,0.6,8.0,2.8,76.0,3900.0,22000.0
4,24111,2017-01-05,0.3,-0.1,3.0,6.2,79.0,15000.0,47000.0


,Unnamed: 0,accident_id,total_accidents,fatal_accidents,injury_accidents,damage_only_accidents,geometry_wkt,geometry,centroid,index_right,...,weekday_2020,weekday_2019,weekday_2018,weekday_2017,location_code,wkt_lng_lat,wkt_lat_lng,longitude,latitude,distance_m
0,0,1,1,0,1,0,POLYGON ((179046.73279999942 309408.8781999983...,"POLYGON ((5.728085069118202 50.77410669272724,...",POINT (179040.2349995231 309458.4569960681),131,...,633,0,830,0,VRA01_0155,POINT(5.164806 52.252951),POINT(52.252951 5.164806),5.164806,52.252951,169007.617336
1,1,2,1,0,0,1,"POLYGON ((177255.8396999985 310240.2259999998,...","POLYGON ((5.702748546852566 50.78165155046857,...",POINT (177249.3590014013 310289.8069981025),131,...,633,0,830,0,VRA01_0155,POINT(5.164806 52.252951),POINT(52.252951 5.164806),5.164806,52.252951,167790.579361
2,2,3,1,0,0,1,POLYGON ((178901.10350000113 311532.4411999993...,"POLYGON ((5.72616005741394 50.79320068551851, ...",POINT (178894.6069976456 311582.0199980425),131,...,633,0,830,0,VRA01_0155,POINT(5.164806 52.252951),POINT(52.252951 5.164806),5.164806,52.252951,166908.586421
3,3,4,1,0,1,0,POLYGON ((178656.49430000037 313910.4211000018...,"POLYGON ((5.722845762480663 50.8145856106313, ...",POINT (178649.9999999999 313960),131,...,633,0,830,0,VRA01_0155,POINT(5.164806 52.252951),POINT(52.252951 5.164806),5.164806,52.252951,164539.720943
4,4,5,2,0,2,0,"POLYGON ((176617.4726999998 313949.6506000012,...","POLYGON ((5.693918115118937 50.81501866838283,...",POINT (176610.9979999998 313999.232),131,...,633,0,830,0,VRA01_0155,POINT(5.164806 52.252951),POINT(52.252951 5.164806),5.164806,52.252951,164032.344449


,Unnamed: 0,Accident_ID,Accident_Date,Time,Hour,Fatalities,Severe_Injuries,Minor_Injuries,Vehicle_Type,Participant_Type,Age,Gender,Municipality,Alcohol_Involved,Day_Type,WVK_ID,shape
0,0,20140010399,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,Amsterdam,NaN,NaN,244380021.0,LINESTRING (122469.24489999935 490087.15410000...
1,1,20140011623,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,Amsterdam,NaN,NaN,242372014.0,LINESTRING (121256.4354000017 486163.425799999...
2,2,20140011930,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,Amsterdam,NaN,NaN,253372001.0,"LINESTRING (126588.02679999918 486233.7511, 12..."
3,3,20140014189,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,Amsterdam,NaN,NaN,241378037.0,LINESTRING (120583.93600000069 489379.40599999...
4,4,20140014197,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,Amsterdam,NaN,NaN,233376127.0,LINESTRING (116948.81799999997 488192.20080000...


In [ ]:
#GeoDataFrame uses WGS84 (lat/lon)
gdf_accidents = gpd.GeoDataFrame(df_accidents, 
                                  geometry=gpd.points_from_xy(df_accidents.longitude, df_accidents.latitude),
                                  crs="EPSG:4326")



In [ ]:
# Create base map
m = folium.Map(location=[52.3702, 4.8952], zoom_start=12)  # Amsterdam


In [ ]:
for _, row in gdf_accidents.iterrows():
    folium.CircleMarker(
        location=[row.geometry.y, row.geometry.x],
        radius=3,
        color="red",
        fill=True,
        fill_opacity=0.5
    ).add_to(m)


In [ ]:
m

In [ ]:
# Group by location
hotspots = gdf_accidents.groupby(["latitude", "longitude"]).size().reset_index(name="count")
top_hotspots = hotspots.nlargest(20, "count")

# Add to map
for _, row in top_hotspots.iterrows():
    folium.CircleMarker(
        location=[row.latitude, row.longitude],
        radius=row["count"],  # bigger circle for more accidents
        color="darkred",
        fill=True,
        fill_opacity=0.6,
        popup=f"{row['count']} accidents"
    ).add_to(m)


In [ ]:
m

In [ ]:
#Accident density Heatmap
# Initialize a folium map centered around average location
m = folium.Map(location=[ gdf_accidents.latitude.mean(),  gdf_accidents.longitude.mean()], zoom_start=11, tiles='CartoDB positron')

# Prepare heatmap data
heat_data = [[row['latitude'], row['longitude']] for index, row in  gdf_accidents.iterrows()]

# Add heatmap layer
HeatMap(heat_data, radius=10, blur=15, max_zoom=1).add_to(m)

m


In [ ]:
#Accident Severity Distribution
severity_df= pd.DataFrame({
    'Fatal': gdf_accidents['fatal_accidents'],
    'Injury': gdf_accidents['injury_accidents'],
    'Damage Only': gdf_accidents['damage_only_accidents']
}).sum()
# Create a vibrant dark green palette
green_palette = sns.dark_palette("#008000", n_colors=3)

# Plot with transparent background and matching text color
plt.figure(figsize=(8, 5), facecolor='none')
ax = severity_df.plot(kind='bar', color=green_palette)

# Style adjustments
plt.title('Distribution of Accident Severity', fontsize=14, color='#008000')
plt.ylabel('Number of Accidents', color='#008000')
plt.xticks(rotation=0, color='#008000')
plt.yticks(color='#008000')
plt.grid(False)
plt.gca().set_facecolor('none')  # Transparent plot background
plt.tight_layout()
plt.show()


In [ ]:
#Relation between bike traffic volume and accident count
gdf_accidents.head()

In [ ]:
# Sum weekday bike volume as a proxy for bike traffic volume

In [ ]:
gdf_accidents['total_weekday_volume'] = (
    gdf_accidents['weekday_2017'] +
    gdf_accidents['weekday_2018'] +
    gdf_accidents['weekday_2019'] +
    gdf_accidents['weekday_2020']
)

# Plot relationship between bike volume and total accidents
plt.figure(figsize=(8, 6))
sns.scatterplot(
    x='total_weekday_volume',
    y='total_accidents',
    data=gdf_accidents,
    alpha=0.6
)
plt.title('Relation Between Bike Volume and Accident Count')
plt.xlabel('Total Weekday Bike Volume (2017-2020)')
plt.ylabel('Total Number of Accidents')
plt.tight_layout()
plt.show()


In [ ]:
# Step 1: Identify top hotspot location
top_location = gdf_accidents.groupby('location_code')['total_accidents'].sum().sort_values(ascending=False).head(1).index[0]

# Step 2: Filter to only that location
hotspot_df = gdf_accidents[gdf_accidents['location_code'] == top_location]

# Step 3: Create a DataFrame for plotting severity by weekday in 2020
severity_by_weekday = pd.DataFrame({
    'Fatal': hotspot_df['fatal_accidents'],
    'Injury': hotspot_df['injury_accidents'],
    'Damage Only': hotspot_df['damage_only_accidents'],
    'Weekday Volume 2020': hotspot_df['weekday_2020']
})

# Melt for Seaborn plotting
severity_melted = severity_by_weekday.melt(
    id_vars='Weekday Volume 2020',
    value_vars=['Fatal', 'Injury', 'Damage Only'],
    var_name='Severity',
    value_name='Accident Count'
)

# Create a sorted order for x-axis (descending weekday volume)
volume_order = (
    severity_melted[['Weekday Volume 2020']]
    .drop_duplicates()
    .sort_values(by='Weekday Volume 2020')
)['Weekday Volume 2020']

# Step 4: Plot with Seaborn using order parameter
plt.figure(figsize=(9, 6), facecolor='none')
sns.barplot(
    data=severity_melted,
    x='Weekday Volume 2020',
    y='Accident Count',
    hue='Severity',
    palette='vlag',
    order=volume_order
)

plt.title(f'Severity of Accidents at Hotspot {top_location}', fontsize=14, color='#008000')
plt.xlabel('Bike Volume on Weekdays (2020)', color='#008000')
plt.ylabel('Number of Accidents', color='#008000')
plt.xticks(color='#008000')
plt.yticks(color='#008000')
plt.legend(title='Severity')
plt.gca().set_facecolor('none')
plt.tight_layout()
plt.show()


In [5]:


# Check for presence of 'shape' column
if 'shape' in bike_data.columns:
    # Extract RD coordinates from LINESTRING
    def extract_rd_coords(linestring):
        match = re.search(r'LINESTRING \(([^,]+)', linestring)
        if match:
            coords = match.group(1).strip().split()
            return float(coords[0]), float(coords[1])
        return None, None

    bike_data[['rd_x', 'rd_y']] = bike_data['shape'].apply(lambda x: pd.Series(extract_rd_coords(x)) if pd.notna(x) else pd.Series([None, None]))

    # Convert to lat/lon using pyproj (EPSG:28992 to EPSG:4326)
    transformer = pyproj.Transformer.from_crs("EPSG:28992", "EPSG:4326", always_xy=True)
    bike_data[['longitude', 'latitude']] = bike_data.apply(lambda row: pd.Series(transformer.transform(row['rd_x'], row['rd_y'])) if pd.notna(row['rd_x']) else pd.Series([None, None]), axis=1)

    # Drop rows without valid coordinates
    bike_data.dropna(subset=['latitude', 'longitude'], inplace=True)

    # Sample for performance
    bike_data_sample = bike_data.head(200)

    # Create map
    m = folium.Map(location=[bike_data_sample['latitude'].mean(), bike_data_sample['longitude'].mean()], zoom_start=12)
    marker_cluster = MarkerCluster().add_to(m)

    for _, row in bike_data_sample.iterrows():
        folium.Marker(
            location=[row['latitude'], row['longitude']],
            popup=f"Accident ID: {row.get('VKL_NUMMER', 'N/A')}"
        ).add_to(marker_cluster)

    m
else:
    "The dataset does not contain a 'shape' column. Please upload the correct version."


# Save and display
m
